### 0.0 Importando bibliotecas


In [2]:
import numpy as np  
import pandas as pd  
import datetime as dt  
import warnings  
import re


import matplotlib.pyplot as plt  
import seaborn as sns  

from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

### 1.0 Extração e Tratamento dos dados


Extraíndo arquivo das informações dos candidatos

In [2]:
df_applicants = pd.read_json('bases/applicants.json', orient='index')

infos_basicas = pd.json_normalize(df_applicants['infos_basicas'])
informacoes_pessoais = pd.json_normalize(df_applicants['informacoes_pessoais'])
informacoes_profissionais = pd.json_normalize(df_applicants['informacoes_profissionais'])
formacao_e_idiomas = pd.json_normalize(df_applicants['formacao_e_idiomas'])
cargo_atual = pd.json_normalize(df_applicants['cargo_atual'])

df_applicants = pd.concat([infos_basicas, informacoes_pessoais, informacoes_profissionais, formacao_e_idiomas, cargo_atual], axis=1)

Extraíndo arquivo das vagas

In [3]:
df_vagas = pd.read_json('bases/vagas.json', orient='index')

df_vagas.reset_index(inplace=True)
df_vagas.rename(columns={"index": "codigo_vaga"}, inplace=True)

df_info_basicas = pd.json_normalize(df_vagas['informacoes_basicas'])
df_perfil_vaga = pd.json_normalize(df_vagas['perfil_vaga']) 
df_beneficios = pd.json_normalize(df_vagas['beneficios'])  

df_vagas = pd.concat([df_vagas[['codigo_vaga']], df_info_basicas, df_perfil_vaga, df_beneficios], axis=1)

Extraíndo arquivo dos candidatos prospectados

In [4]:
df_prospects = pd.read_json('bases/prospects.json', orient='index')

df_prospects.reset_index(inplace=True)
df_prospects.rename(columns={"index": "codigo_vaga"}, inplace=True)

df_prospects = df_prospects.explode('prospects').reset_index(drop=True)
df_prospects_detalhado = pd.json_normalize(df_prospects['prospects'])  # Removido o .add_prefix()
df_prospects = pd.concat([df_prospects.drop(columns=['prospects']), df_prospects_detalhado], axis=1)

### 1.1 Tratando a tabela df_applicants

Remove colunas duplicadas resultantes da concatenação dos blocos normalizados de dados dos candidatos.


In [5]:
df_applicants = df_applicants.loc[:, ~df_applicants.columns.duplicated(keep='first')]

In [6]:
df_applicants.columns

Index(['telefone_recado', 'telefone', 'objetivo_profissional', 'data_criacao',
       'inserido_por', 'email', 'local', 'sabendo_de_nos_por',
       'data_atualizacao', 'codigo_profissional', 'nome', 'data_aceite', 'cpf',
       'fonte_indicacao', 'email_secundario', 'data_nascimento',
       'telefone_celular', 'sexo', 'estado_civil', 'pcd', 'endereco', 'skype',
       'url_linkedin', 'facebook', 'download_cv', 'titulo_profissional',
       'area_atuacao', 'conhecimentos_tecnicos', 'certificacoes',
       'outras_certificacoes', 'remuneracao', 'nivel_profissional',
       'qualificacoes', 'experiencias', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'outro_idioma', 'instituicao_ensino_superior',
       'cursos', 'ano_conclusao', 'outro_curso', 'id_ibrati',
       'email_corporativo', 'cargo_atual', 'projeto_atual', 'cliente',
       'unidade', 'data_admissao', 'data_ultima_promocao',
       'nome_superior_imediato', 'email_superior_imediato', 'empresa', 'cargo',
       '

Selecionando apenas as colunas desejadas

In [7]:
colunas_applicants = [
    "codigo_profissional","objetivo_profissional",
    "pcd", "area_atuacao",
    "conhecimentos_tecnicos","certificacoes","outras_certificacoes",
    "nivel_profissional", "nivel_academico", "nivel_ingles",
    "nivel_espanhol","outro_idioma","cursos","cargo_atual","local"
]

df_applicants = df_applicants[colunas_applicants]

In [8]:
df_applicants.shape

(42483, 15)

Validando dados Nulls e brancos

In [9]:
df_applicants.isnull().sum()

codigo_profissional           0
objetivo_profissional         0
pcd                           0
area_atuacao                  0
conhecimentos_tecnicos        0
certificacoes                 0
outras_certificacoes          0
nivel_profissional            0
nivel_academico               0
nivel_ingles                  0
nivel_espanhol                0
outro_idioma                  0
cursos                    35274
cargo_atual               41539
local                         0
dtype: int64

In [10]:
(df_applicants == "").sum()

codigo_profissional           0
objetivo_profissional     14416
pcd                       36161
area_atuacao              35731
conhecimentos_tecnicos    39129
certificacoes             41954
outras_certificacoes      41584
nivel_profissional        42402
nivel_academico           34388
nivel_ingles              35638
nivel_espanhol            35795
outro_idioma                  0
cursos                     1070
cargo_atual                 546
local                     25474
dtype: int64

In [11]:
percentuais_vazios = (df_applicants == "").sum() / df_applicants.shape[0]
print(percentuais_vazios.round(2))

codigo_profissional       0.00
objetivo_profissional     0.34
pcd                       0.85
area_atuacao              0.84
conhecimentos_tecnicos    0.92
certificacoes             0.99
outras_certificacoes      0.98
nivel_profissional        1.00
nivel_academico           0.81
nivel_ingles              0.84
nivel_espanhol            0.84
outro_idioma              0.00
cursos                    0.03
cargo_atual               0.01
local                     0.60
dtype: float64


Para garantir a qualidade dos dados e remover entradas indesejadas ou inválidas, criamos uma função que limpa as colunas de informações, removendo valores como e-mails, datas, e dados vazios. O objetivo é manter apenas entradas com caracteres alfabéticos relevantes para a análise.

#### Lógica da Função:
- **E-mails**: Se a entrada for um endereço de e-mail (identificado por um padrão regex), ela será removida.
- **Datas**: Caso o valor seja reconhecido como uma data válida, ele será descartado.
- **Vazios e ruídos**: Entradas vazias, com pontos, hífens ou espaços em branco também são removidas.
- **Caracteres alfabéticos**: A função mantém apenas entradas que contêm caracteres alfabéticos (a, b, c, ... Z).
  

In [12]:
def filtrar_somente_caracteres(df, coluna):
    def limpar_objetivo(obj):
        obj = str(obj).strip() 
        
        if pd.isna(obj) or obj in ["", ".", "-", " "]:
            return ""
             
        if re.match(r"^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$", obj):
            return ""
        
        if pd.to_datetime(obj, errors='coerce') is not pd.NaT:
            return ""
        
        if not re.search(r"[a-zA-Z]", obj):
            return ""
        
        obj = obj.replace('nan', '') 
        
        return obj  

    df[coluna] = df[coluna].apply(limpar_objetivo)
    return df


In [13]:
colunas_para_limpar = [
    "objetivo_profissional", "pcd", "area_atuacao", "conhecimentos_tecnicos",
    "certificacoes", "outras_certificacoes", "nivel_profissional", "nivel_academico",
    "nivel_ingles", "nivel_espanhol", "outro_idioma", "cursos", "cargo_atual", "local"
]

for coluna in colunas_para_limpar:
    df_applicants = filtrar_somente_caracteres(df_applicants, coluna)
    

Retirando todas as linhas onde o objetivo profissional for vazio, pois usaremos essa coluna como nossa principal, por possuir menos dados relevantes vazios

In [14]:
df_applicants = df_applicants[df_applicants['objetivo_profissional'] != '']

In [15]:
df_applicants.shape

(27995, 15)

In [16]:
(df_applicants == "").sum()

codigo_profissional           0
objetivo_profissional         0
pcd                       22150
area_atuacao              21761
conhecimentos_tecnicos    24777
certificacoes             27508
outras_certificacoes      27150
nivel_profissional        27931
nivel_academico           20455
nivel_ingles              21734
nivel_espanhol            21837
outro_idioma              26681
cursos                    22256
cargo_atual               27639
local                     11665
dtype: int64

Para facilitar o processamento de texto, especialmente em tarefas de NLP (Natural Language Processing), criamos uma função que concatena várias colunas em um único campo. Isso permite que possamos trabalhar com informações de maneira mais eficiente, sem a necessidade de analisar colunas individualmente.

#### Lógica:
- **Filtro de valores vazios**: Quando uma coluna contém um valor em branco, ela é ignorada na concatenação da linha. Isso é importante para evitar a inclusão de dados irrelevantes (ruídos) no modelo, o que pode prejudicar a qualidade da análise.
- **Concatenando as colunas**: As colunas selecionadas para cada grupo de informações são unidas em uma string, com um separador " | ", de modo a manter as informações organizadas e separadas.


In [17]:
def criar_grupos_concatenados(df, id_coluna, grupos_colunas):
    dfs_resultado = {}

    for nome_grupo, colunas in grupos_colunas.items():
        df_temp = df[[id_coluna] + colunas].copy()

        def concatenar_linha(row):
            textos_validos = [
                f"{coluna}: {str(row[coluna]).strip()}"
                for coluna in colunas
                if str(row[coluna]).strip() != ""
            ]
            return ' | '.join(textos_validos)

        df_temp[f"perfil_concatenado_{nome_grupo}"] = df_temp.apply(concatenar_linha, axis=1)

        df_temp = df_temp[df_temp[f"perfil_concatenado_{nome_grupo}"] != ""]

        dfs_resultado[nome_grupo] = df_temp[[id_coluna, f"perfil_concatenado_{nome_grupo}"]]

    return dfs_resultado


Para aplicar diferentes pesos nas habilidades durante o modelo de similaridade, separamos as colunas de dados em três grupos distintos: **Hard Skills**, **Educação** e **Personal**. Esta organização facilita a aplicação de pesos personalizados em cada grupo, dependendo da sua importância no modelo.

#### Etapas:

1. **Definição dos grupos**:
   - **Hard Skills**: Inclui informações como objetivos profissionais, área de atuação, conhecimentos técnicos, cargo atual, etc.
   - **Educação**: Engloba certificações, níveis acadêmicos e fluência em idiomas.
   - **Pessoais**: Contém dados sobre pessoas com deficiência (PCD) e a localização dos candidatos.

2. **Criação de grupos concatenados**:
   Para cada grupo, concatenamos as informações das colunas relevantes. Caso uma coluna contenha dados vazios ou inválidos, esses valores são excluídos, como feito no passo anterior.

3. **Mesclagem dos DataFrames**:
   Os DataFrames de cada grupo são mesclados em um único DataFrame `df_unico`, utilizando `outer join` para garantir que todas as informações sejam mantidas, mesmo que algumas colunas não possuam valores para certos candidatos.

4. **Preenchimento de valores ausentes**:
      Utilizamos `fillna('')` para substituir valores `NaN` por strings vazias, garantindo que o modelo de similaridade não seja afetado por dados faltantes.

In [18]:
hard_skills = [
    "objetivo_profissional",
    "area_atuacao",
    "conhecimentos_tecnicos",
    "nivel_profissional",
    "cargo_atual",
    "cursos"
]

education = [
    "certificacoes",
    "outras_certificacoes",
    "nivel_academico",
    "nivel_ingles",
    "nivel_espanhol",
    "outro_idioma"
]

personal = [
    "pcd",
    "local"
]

grupos_colunas = {
    "hard_skills": hard_skills,
    "education": education,
    "personal": personal
}

dfs_por_grupo = criar_grupos_concatenados(df_applicants, "codigo_profissional", grupos_colunas)

df_hard_skills = dfs_por_grupo["hard_skills"]
df_education = dfs_por_grupo["education"]
df_personal = dfs_por_grupo["personal"]

df_unico = df_hard_skills.merge(df_education, on="codigo_profissional", how="outer")
df_unico = df_unico.merge(df_personal, on="codigo_profissional", how="outer")

df_unico.fillna('',inplace=True)

In [19]:
(df_unico == "").sum()

codigo_profissional                   0
perfil_concatenado_hard_skills        0
perfil_concatenado_education      20428
perfil_concatenado_personal       11664
dtype: int64

In [20]:
df_unico.dtypes

codigo_profissional               object
perfil_concatenado_hard_skills    object
perfil_concatenado_education      object
perfil_concatenado_personal       object
dtype: object

In [21]:
df_unico['codigo_profissional'] = df_unico['codigo_profissional'].astype(int)

###  1.2 Tratando a tabela df_prospects

In [22]:
df_prospects.columns

Index(['codigo_vaga', 'titulo', 'modalidade', 'nome', 'codigo',
       'situacao_candidado', 'data_candidatura', 'ultima_atualizacao',
       'comentario', 'recrutador'],
      dtype='object')

In [23]:
df_prospects.shape

(56702, 10)

In [24]:
df_prospects.isnull().sum()

codigo_vaga              0
titulo                   0
modalidade               0
nome                  2943
codigo                2943
situacao_candidado    2943
data_candidatura      2943
ultima_atualizacao    2943
comentario            2943
recrutador            2943
dtype: int64

In [25]:
(df_prospects == "").sum()

codigo_vaga               0
titulo                 2943
modalidade            55019
nome                      0
codigo                    0
situacao_candidado        0
data_candidatura          0
ultima_atualizacao     3913
comentario            39201
recrutador                0
dtype: int64

Extraindo apenas colunas que achamos relevantes

In [26]:
df_prospects = df_prospects[["codigo","codigo_vaga","titulo","situacao_candidado"]]

In [27]:
df_prospects.isnull().sum()

codigo                2943
codigo_vaga              0
titulo                   0
situacao_candidado    2943
dtype: int64

Removemos as linhas em branco, pois identificamos que a coluna 'codigo' contém esses valores vazios e consideramos essa coluna essencial para o nosso processo.

In [28]:
df_prospects.dropna(inplace=True)

In [29]:
df_prospects.shape

(53759, 4)

In [30]:
(df_prospects == "").sum()

codigo                0
codigo_vaga           0
titulo                0
situacao_candidado    0
dtype: int64

Neste passo, criamos uma nova coluna chamada `situacao_candidado` no DataFrame `df_prospects`, com valores binários (1 ou 0), onde:
- **1**: para candidatos com status de "Aprovado" (ou status relacionados, como "Proposta Aceita" ou "Contratado pela Decision").
- **0**: para os candidatos que possuem status de "Não Aprovado" ou qualquer outro status que não indique aprovação.

#### Etapas:

1. **Definição dos status**: 
   - `status_aprovado`: Contém todos os status que indicam que o candidato foi aprovado.
   - `status_negado`: Inclui os status que indicam que o candidato não foi aprovado.

2. **Limpeza do status**: A função `str.strip()` é aplicada para remover espaços extras antes ou depois do texto em cada linha da coluna `situacao_candidado`.

3. **Aplicação da lógica**: A função `apply` é usada para aplicar a condição:
   - Se o valor da coluna `situacao_candidado` estiver na lista `status_aprovado`, o valor será 1.
   - Caso contrário, o valor será 0.

In [31]:
df_prospects['situacao_candidado'].unique()

array(['Encaminhado ao Requisitante', 'Contratado pela Decision',
       'Desistiu', 'Documentação PJ', 'Não Aprovado pelo Cliente',
       'Prospect', 'Não Aprovado pelo RH', 'Aprovado',
       'Não Aprovado pelo Requisitante', 'Inscrito', 'Entrevista Técnica',
       'Em avaliação pelo RH', 'Contratado como Hunting',
       'Desistiu da Contratação', 'Entrevista com Cliente',
       'Documentação CLT', 'Recusado', 'Documentação Cooperado',
       'Sem interesse nesta vaga', 'Encaminhar Proposta',
       'Proposta Aceita'], dtype=object)

In [32]:
status_aprovado = [
    'Aprovado',
    'Contratado pela Decision',
    'Contratado como Hunting',
    'Proposta Aceita',
    'Encaminhar Proposta'
]

status_negado = [
    'Não Aprovado pelo Cliente',
    'Não Aprovado pelo RH',
    'Não Aprovado pelo Requisitante',
    'Desistiu',
    'Desistiu da Contratação',
    'Recusado',
    'Sem interesse nesta vaga'
]

df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].str.strip()

df_prospects['situacao_candidado'] = df_prospects['situacao_candidado'].apply(
    lambda x: 1 if x in status_aprovado else 0
)


In [33]:
df_prospects['situacao_candidado'].value_counts()

situacao_candidado
0    50563
1     3196
Name: count, dtype: int64

In [34]:
df_prospects_concatenado = df_prospects[['codigo','situacao_candidado']]

In [35]:
df_prospects_concatenado.dtypes

codigo                object
situacao_candidado     int64
dtype: object

In [36]:
df_prospects_concatenado.rename(columns={'codigo': 'codigo_profissional'}, inplace=True)
df_prospects_concatenado['codigo_profissional'] = df_prospects_concatenado['codigo_profissional'].astype(int)

Criamos um dataframe para excluir os candidatos já aprovados, pois entendemos que não faz sentido incluí-los no retorno, dado que estamos focados na busca por novas contratações.

In [37]:
deletar_aprovados = df_prospects_concatenado[df_prospects_concatenado['situacao_candidado'] == 1]['codigo_profissional']

A tabela Prospects contém dados duplicados, pois um mesmo candidato pode estar vinculado a mais de uma vaga de prospecção. Para lidar com isso, contabilizamos o número de prospecções associadas a cada candidato e garantimos que cada um seja tratado de forma única. Dessa forma, conseguimos incorporar essa informação ao nosso modelo de similaridade.

In [38]:
df_prospects_tratado = df_prospects_concatenado[df_prospects_concatenado['situacao_candidado'] == 0].groupby(by='codigo_profissional').count().reset_index()

In [39]:
df_prospects_tratado.rename(columns={'situacao_candidado':'quantidade_prospect'},inplace=True)

Neste trecho, realizamos um left join para priorizar a tabela de candidatos, uma vez que entendemos que nem todos os candidatos foram prospectados.

In [40]:
df_merge = pd.merge(df_unico, df_prospects_tratado, on=['codigo_profissional'], how='left')

In [41]:
df_merge = df_merge[~df_merge['codigo_profissional'].isin(deletar_aprovados)]

In [42]:
df_merge['quantidade_prospect'].fillna(0,inplace=True)

In [43]:
df_merge['quantidade_prospect'] = df_merge['quantidade_prospect'].astype(int)

In [44]:
df_merge.nunique()

codigo_profissional               27012
perfil_concatenado_hard_skills    10808
perfil_concatenado_education       1802
perfil_concatenado_personal         997
quantidade_prospect                  35
dtype: int64

Realizamos a separação dos dados em DataFrames distintos, de acordo com o tipo de skill, para que os pesos pudessem ser aplicados de forma individualizada em cada grupo.

In [45]:
df_hard_skills = df_merge[["codigo_profissional", "perfil_concatenado_hard_skills", "quantidade_prospect"]].copy()

df_education = df_merge[["codigo_profissional", "perfil_concatenado_education"]].copy()

df_personal = df_merge[["codigo_profissional", "perfil_concatenado_personal"]].copy()

### 1.3 Tratando a tabela df_vagas

In [46]:
df_vagas.columns

Index(['codigo_vaga', 'data_requicisao', 'limite_esperado_para_contratacao',
       'titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
       'empresa_divisao', 'requisitante', 'analista_responsavel',
       'tipo_contratacao', 'prazo_contratacao', 'objetivo_vaga',
       'prioridade_vaga', 'origem_vaga', 'superior_imediato', 'nome',
       'telefone', 'data_inicial', 'data_final', 'nome_substituto', 'pais',
       'estado', 'cidade', 'bairro', 'regiao', 'local_trabalho',
       'vaga_especifica_para_pcd', 'faixa_etaria', 'horario_trabalho',
       'nivel profissional', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'outro_idioma', 'areas_atuacao',
       'principais_atividades', 'competencia_tecnicas_e_comportamentais',
       'demais_observacoes', 'viagens_requeridas', 'equipamentos_necessarios',
       'habilidades_comportamentais_necessarias', 'valor_venda',
       'valor_compra_1', 'valor_compra_2'],
      dtype='object')

In [47]:
colunas_vagas = [
    'codigo_vaga',
    'titulo_vaga',
    'vaga_especifica_para_pcd',
    'nivel profissional',
    'nivel_academico',
    'nivel_ingles',
    'nivel_espanhol',
    'outro_idioma',
    'areas_atuacao',
    'principais_atividades',
    'competencia_tecnicas_e_comportamentais',
    'estado',
    'cidade'
]

df_vagas = df_vagas[colunas_vagas]

Nesses blocos tratamos algumas incosistências nas colunas

In [48]:
df_vagas['titulo_vaga'] = df_vagas['titulo_vaga'].str.replace(r'\b\w*\d+\s*-\s*', '', regex=True).str.strip()
df_vagas['titulo_vaga'] = df_vagas['titulo_vaga'].str.replace(r'\s*-\s*\d+$', '', regex=True).str.strip()

In [49]:
df_vagas['principais_atividades'] = (
    df_vagas['principais_atividades']
    .str.replace(r'\n+', ' ', regex=True)   
    .str.replace('•', '')
    .str.replace('', '')                  
    .str.strip()
)


In [50]:
df_vagas['competencia_tecnicas_e_comportamentais'] = (
    df_vagas['competencia_tecnicas_e_comportamentais']
    .str.replace(r'\n+', ' ', regex=True)   
    .str.replace('•', '')
    .str.replace('', '')                  
    .str.strip()
)

In [51]:
df_vagas.isnull().sum()

codigo_vaga                               0
titulo_vaga                               0
vaga_especifica_para_pcd                  0
nivel profissional                        0
nivel_academico                           0
nivel_ingles                              0
nivel_espanhol                            0
outro_idioma                              0
areas_atuacao                             0
principais_atividades                     0
competencia_tecnicas_e_comportamentais    0
estado                                    0
cidade                                    0
dtype: int64

In [52]:
(df_vagas == "").sum()

codigo_vaga                                   0
titulo_vaga                                   0
vaga_especifica_para_pcd                   1948
nivel profissional                            0
nivel_academico                               0
nivel_ingles                                  0
nivel_espanhol                             1304
outro_idioma                              13708
areas_atuacao                                15
principais_atividades                         3
competencia_tecnicas_e_comportamentais        6
estado                                       11
cidade                                       51
dtype: int64

In [53]:
df_vagas.rename(columns={"nivel profissional":"nivel_profissional"},inplace=True)

Utilizamos este método para concatenar as colunas do df_vagas, com o objetivo de prepará-las para o processamento de NLP.

In [54]:
hard_skills_vagas = [
    "titulo_vaga"
    ,"areas_atuacao"
    ,"principais_atividades"
    ,"competencia_tecnicas_e_comportamentais"
    ,"nivel_profissional"
]

education_vagas = [
    "nivel_academico"
    ,"nivel_ingles"
    ,"nivel_espanhol"
    ,"outro_idioma"  
]

personal_vagas = [
    "vaga_especifica_para_pcd"
    ,"estado"
    ,"cidade"
]

grupos_colunas_vagas = {
    "hard_skills_vagas": hard_skills_vagas,
    "education_vagas": education_vagas,
    "personal_vagas": personal_vagas
}

dfs_por_grupo_vaga = criar_grupos_concatenados(df_vagas, "codigo_vaga", grupos_colunas_vagas)


df_hard_skills_vagas = dfs_por_grupo_vaga["hard_skills_vagas"]
df_education_vagas = dfs_por_grupo_vaga["education_vagas"]
df_personal_vagas = dfs_por_grupo_vaga["personal_vagas"]

### 2.0 Aplicando o processo de NLP

 Este bloco tem como objetivo transformar os textos concatenados em vetores de embeddings utilizando o modelo all-MiniLM-L12-v2 da biblioteca SentenceTransformer.
 Ao converter os textos das colunas em representações vetoriais, conseguimos capturar o contexto semântico de cada entrada.
 Esses vetores serão utilizados posteriormente para calcular a similaridade entre candidatos e vagas, viabilizando uma análise mais precisa de compatibilidade.

Removemos as linhas em branco desses dois DataFrames, pois, como foram extraídas do df_merge, já estavam nulas — indicando que o candidato não possuía essas informações preenchidas. Essa limpeza foi feita com o objetivo de reduzir o custo computacional no processamento dos embeddings, eliminando os registros irrelevantes antes da execução do método.

In [55]:
df_personal = df_personal[df_personal['perfil_concatenado_personal'] != '']
df_education = df_education[df_education['perfil_concatenado_education'] != '']

A função `gerar_embeddings` tem como objetivo transformar informações textuais de uma coluna de um DataFrame em vetores numéricos (embeddings)

#### Etapas:
1. **Conversão para String**: A coluna de texto é convertida para tipo string para garantir que todos os dados sejam manipulados de forma consistente.
   
2. **Geração dos Embeddings**: Utilizando o modelo pré-treinado `all-MiniLM-L12-v2`, os textos são transformados em vetores de alta dimensionalidade. A função `encode` da biblioteca **SentenceTransformer** gera esses vetores.

3. **Criação de DataFrame para Embeddings**: A função cria um DataFrame contendo os embeddings gerados, nomeando cada coluna com base no prefixo fornecido e no índice da dimensão do embedding.

4. **Concatenação com o DataFrame Original**: Os embeddings gerados são concatenados com o DataFrame original para que cada candidato possua seu vetor de embeddings associado.


In [56]:
model = SentenceTransformer('all-MiniLM-L12-v2')

def gerar_embeddings(df, coluna_texto, prefixo_nome_embedding):
    df = df.copy()
    df[coluna_texto] = df[coluna_texto].astype(str)
    
    embeddings = model.encode(df[coluna_texto].tolist(), show_progress_bar=True)
    embeddings_df = pd.DataFrame(
        embeddings,
        columns=[f'{prefixo_nome_embedding}_embedding_{i}' for i in range(embeddings.shape[1])]
    )
    
    return pd.concat([df.reset_index(drop=True), embeddings_df], axis=1)

df_hard_skills_embed = gerar_embeddings(df_hard_skills, 'perfil_concatenado_hard_skills', 'hard')
df_education_embed = gerar_embeddings(df_education, 'perfil_concatenado_education', 'edu')
df_personal_embed = gerar_embeddings(df_personal, 'perfil_concatenado_personal', 'pers')


Batches: 100%|██████████| 486/486 [00:37<00:00, 13.06it/s]


Neste dicionário, definimos os pesos atribuídos a cada tipo de informação (por exemplo, hard skills, educação, características pessoais), com o objetivo de equilibrar a influência de cada uma durante o cálculo de similaridade. Essa ponderação evita que determinados grupos de atributos dominem a análise, garantindo resultados mais justos e representativos.

In [57]:
pesos = {
    'hard_skills': 0.6,
    'education': 0.3,
    'personal': 0.1
}

In [58]:
def aplicar_peso(df_embed, peso, prefixo):
    cols_embed = [col for col in df_embed.columns if col.startswith(prefixo)]
    df_embed[cols_embed] = df_embed[cols_embed] * peso
    return df_embed

In [59]:
df_hard_skills_embed = aplicar_peso(df_hard_skills_embed, pesos['hard_skills'], 'hard')
df_education_embed = aplicar_peso(df_education_embed, pesos['education'], 'edu')
df_personal_embed = aplicar_peso(df_personal_embed, pesos['personal'], 'pers')

Após aplicar os pesos em todos os DataFrames de embeddings, realizamos a união desses dados em um único DataFrame consolidado. Essa fusão nos permite utilizar a representação vetorial completa de cada candidato para o cálculo de similaridade de forma integrada e balanceada.

In [60]:
df_temp = df_hard_skills_embed.merge(
    df_education_embed, on='codigo_profissional', how='left', suffixes=('', '_edu')
)

df_final = df_temp.merge(
    df_personal_embed, on='codigo_profissional', how='left', suffixes=('', '_pers')
)

Removemos algumas colunas indesejadas

In [61]:
df_final_candidatos = df_final[df_final.columns[~df_final.columns.str.contains("perfil_concatenado", case=False)]]

Como resultado do processo de junção (left join) entre os DataFrames de embeddings, muitas colunas apresentam valores nulos devido à ausência de informações em determinados perfis. Para manter a consistência dos vetores e viabilizar o cálculo da similaridade do cosseno, optamos por substituir os valores nulos por zero. Essa abordagem é adequada, pois a presença de zeros em embeddings simplesmente indica ausência de informação naquela dimensão, sem comprometer a integridade do cálculo de similaridade.

In [62]:
df_final_candidatos.fillna(0,inplace=True)

Aqui aplicamos o mesmo processo que efetuamos nos candidatos, porém com as vagas

In [63]:
df_hard_skills_embed_vaga = gerar_embeddings(df_hard_skills_vagas, 'perfil_concatenado_hard_skills_vagas', 'vhard')
df_education_embed_vaga = gerar_embeddings(df_education_vagas, 'perfil_concatenado_education_vagas', 'vedu')
df_personal_embed_vaga = gerar_embeddings(df_personal_vagas, 'perfil_concatenado_personal_vagas', 'vpers')

Batches: 100%|██████████| 441/441 [01:04<00:00,  6.88it/s]


Aplicamos os mesmos pesos, para ter a mesma magnitude

In [64]:
df_hard_skills_vaga_embed = aplicar_peso(df_hard_skills_embed_vaga, pesos['hard_skills'], 'vhard')
df_education_vaga_embed = aplicar_peso(df_education_embed_vaga, pesos['education'], 'vedu')
df_personal_vaga_embed = aplicar_peso(df_personal_embed_vaga, pesos['personal'], 'vpers')

Realizamos o left join e a mesma tratativa dos valores nulls

In [65]:
df_vagas_embed = df_hard_skills_vaga_embed.merge(
    df_education_vaga_embed, on='codigo_vaga', how='left'
).merge(
    df_personal_vaga_embed, on='codigo_vaga', how='left'
)

df_vagas_embed.fillna(0, inplace=True)


Retiramos algumas colunas indesejadas

In [66]:
df_final_vagas = df_vagas_embed[df_vagas_embed.columns[~df_vagas_embed.columns.str.contains("perfil_concatenado", case=False)]]

## 3.0 Função de Recomendação de Candidatos com Similaridade de Cosseno

Esta função realiza a recomendação de candidatos com base na similaridade de cosseno entre os vetores de embeddings completos (hard skills, formação e perfil pessoal) de candidatos e da vaga especificada.

### Parâmetros de entrada:
- `codigo_vaga_input`: Código da vaga usada como referência.
- `df_candidatos_final`: DataFrame consolidado com todos os embeddings dos candidatos.
- `df_vagas_final`: DataFrame consolidado com todos os embeddings das vagas.
- `top_n`: Número de candidatos mais similares a serem retornados (padrão = 5).

### Etapas:
1. Filtra a vaga correspondente ao código informado.
2. Seleciona todas as colunas que contenham a palavra `'embedding'` no nome, garantindo que todos os vetores (hard, education e personal) sejam considerados.
3. Extrai o vetor completo da vaga e os vetores dos candidatos.
4. Calcula a similaridade de cosseno entre o vetor da vaga e os vetores dos candidatos.
5. Adiciona o score de similaridade ao DataFrame de candidatos.
6. Ordena os candidatos por `score_cosine` de forma decrescente.
7. Retorna os `top_n` candidatos mais similares, incluindo o `codigo_profissional`, o `score_cosine` e a `quantidade_prospect`.


In [67]:
def recomendar_candidatos_cosine(codigo_vaga_input, df_candidatos_final, df_vagas_final, top_n=5):

    vaga_embed = df_vagas_final[df_vagas_final['codigo_vaga'] == codigo_vaga_input]
    if vaga_embed.empty:
        return "Vaga não encontrada."

    embedding_cols_vaga = [col for col in df_vagas_final.columns if 'embedding' in col]
    embedding_cols_candidatos = [col for col in df_candidatos_final.columns if 'embedding' in col]

    vaga_vector = vaga_embed[embedding_cols_vaga].values

    candidatos_disponiveis = df_candidatos_final

    candidatos_vectors = candidatos_disponiveis[embedding_cols_candidatos].values

    sim_scores = cosine_similarity(candidatos_vectors, vaga_vector).flatten()
    candidatos_disponiveis['score_cosine'] = sim_scores

    candidatos_disponiveis['score_cosine_percentual'] = (candidatos_disponiveis['score_cosine'] * 100).round(2).astype(str) + '%'

    df_applicants['codigo_profissional'] = df_applicants['codigo_profissional'].astype(int)

    candidatos_com_info = candidatos_disponiveis.merge(
        df_applicants[['codigo_profissional', 'objetivo_profissional']], 
        on='codigo_profissional', 
        how='left'
    )

    return candidatos_com_info.sort_values('score_cosine', ascending=False).head(top_n)[
        ['codigo_profissional','objetivo_profissional','score_cosine_percentual','quantidade_prospect']
    ]


In [91]:
codigo_vaga = 5180


result = recomendar_candidatos_cosine(codigo_vaga, df_final_candidatos, df_final_vagas, top_n=5)

result

,codigo_profissional,objetivo_profissional,score_cosine_percentual,quantidade_prospect
3919,14320,Desenvolvedor Web,82.21%,1
2135,12,Desenvolvedor Web,81.75%,0
6421,21126,Desenvolvedor .NET,81.07%,4
6066,20033,"Analista de Sistemas, Full Stack .Net Core, An...",80.97%,6
6063,20027,Web Designer ou Desenvolvedor,80.76%,1
